In [1]:
import numpy as np
import random
from timeit import default_timer as timer
from datetime import datetime

In [2]:
def generate_cities(file):
    file1 = open(file, 'r')
    start = False
    cities_dict = {}

    while True:

        # Get next line from file
        line = file1.readline()

        # if line is empty
        # end of file is reached
        if not line:
            break

        if start:
            citystr, xstr, ystr = line.rstrip().split()
            city = citystr
            x = float(xstr)
            y = float(ystr)
            cities_dict[city] = [x, y]

        if line == "NODE_COORD_SECTION\n":
            start = True
    file1.close()
    return cities_dict

In [3]:
# Function to compute the distance between two points
def compute_city_distance_coordinates(a,b):
    return ((a[0]-b[0])**2+(a[1]-b[1])**2)**0.5

def compute_city_distance_names(city_a, city_b, cities_dict):
    return compute_city_distance_coordinates(cities_dict[city_a], cities_dict[city_b])

In [12]:
cities_dict = generate_cities("Oman1979.tsp")
names_list = list(cities_dict.keys())
names_list = np.array(names_list)
mutation_rate = 0.5
n_population = 50
n_cities = 1979

In [13]:
# First step: Create the first population set
def genesis(city_list, n_population):

    population_set = []
    for i in range(n_population):
        #Randomly generating a new solution
        sol_i = city_list[np.random.choice(list(range(n_cities)), n_cities, replace=False)]
        population_set.append(sol_i)
    return np.array(population_set)

population_set = genesis(names_list, n_population)
population_set

array([['1690', '1816', '1671', ..., '989', '1601', '575'],
       ['1490', '1549', '567', ..., '639', '590', '318'],
       ['1813', '387', '1534', ..., '562', '1111', '223'],
       ...,
       ['68', '290', '1340', ..., '1392', '1826', '677'],
       ['1571', '558', '880', ..., '548', '616', '902'],
       ['402', '1410', '1846', ..., '571', '1016', '784']], dtype='<U4')

In [14]:
def fitness_eval(city_list, cities_dict):
    total = 0
    for i in range(n_cities-1):
        a = city_list[i]
        b = city_list[i+1]
        total += compute_city_distance_names(a,b, cities_dict)
    return total

In [15]:
def get_all_fitnes(population_set, cities_dict):
    fitnes_list = np.zeros(n_population)

    #Looping over all solutions computing the fitness for each solution
    for i in  range(n_population):
        fitnes_list[i] = fitness_eval(population_set[i], cities_dict)

    return fitnes_list

fitnes_list = get_all_fitnes(population_set,cities_dict)
fitnes_list

array([5567746.82338536, 5550015.31835226, 5576919.00674863,
       5618779.19789325, 5605254.85630298, 5585011.13786958,
       5616537.97505412, 5655017.6911081 , 5547524.50059884,
       5718403.5996448 , 5617349.14095403, 5602113.88186625,
       5568988.34666473, 5574479.14069316, 5615045.43048146,
       5583850.49562488, 5498096.71621259, 5582191.17622943,
       5678560.7381553 , 5550405.85481709, 5585233.27900556,
       5653397.39413585, 5543138.34803821, 5605901.36094158,
       5741690.5357483 , 5621377.99185804, 5676388.89768901,
       5557378.51302775, 5724272.3647379 , 5601649.6745777 ,
       5615742.99211273, 5731003.48811874, 5526462.24212038,
       5608650.59583906, 5602838.3686553 , 5537257.50315036,
       5527669.20817445, 5603683.46311298, 5668838.18473859,
       5544427.44233304, 5546953.21865048, 5579119.73951787,
       5661378.69525695, 5596271.86594889, 5669378.30236529,
       5599754.698764  , 5663940.68406713, 5698250.06202153,
       5616001.80770674,

In [16]:
def progenitor_selection(population_set,fitnes_list):
    total_fit = fitnes_list.sum()
    prob_list = fitnes_list/total_fit
    
    #Notice there is the chance that a progenitor. mates with oneself
    progenitor_list_a = np.random.choice(list(range(len(population_set))), len(population_set),p=prob_list, replace=True)
    progenitor_list_b = np.random.choice(list(range(len(population_set))), len(population_set),p=prob_list, replace=True)
    
    progenitor_list_a = population_set[progenitor_list_a]
    progenitor_list_b = population_set[progenitor_list_b]
    
    
    return np.array([progenitor_list_a,progenitor_list_b])


progenitor_list = progenitor_selection(population_set,fitnes_list)

In [17]:
def mate_progenitors(prog_a, prog_b):
    offspring = prog_a[0:5]

    for city in prog_b:

        if not city in offspring:
            offspring = np.concatenate((offspring,[city]))

    return offspring
            
    
    
def mate_population(progenitor_list):
    new_population_set = []
    for i in range(progenitor_list.shape[1]):
        prog_a, prog_b = progenitor_list[0][i], progenitor_list[1][i]
        offspring = mate_progenitors(prog_a, prog_b)
        new_population_set.append(offspring)
        
    return new_population_set

new_population_set = mate_population(progenitor_list)

In [18]:
def mutate_offspring(offspring):
    for q in range(int(n_cities*mutation_rate)):
        a = np.random.randint(0,n_cities)
        b = np.random.randint(0,n_cities)

        offspring[a], offspring[b] = offspring[b], offspring[a]

    return offspring
    
    
def mutate_population(new_population_set):
    mutated_pop = []
    for offspring in new_population_set:
        mutated_pop.append(mutate_offspring(offspring))
    return mutated_pop

mutated_pop = mutate_population(new_population_set)

In [19]:
best_solution = [-1,np.inf,np.array([])]
start = timer()
for i in range(500):
    if i%100==0: print(i, fitnes_list.min(), fitnes_list.mean(), datetime.now().strftime("%d/%m/%y %H:%M"))
    fitnes_list = get_all_fitnes(mutated_pop,cities_dict)
    
    #Saving the best solution
    if fitnes_list.min() < best_solution[1]:
        best_solution[0] = i
        best_solution[1] = fitnes_list.min()
        best_solution[2] = np.array(mutated_pop)[fitnes_list.min() == fitnes_list]
    
    progenitor_list = progenitor_selection(population_set,fitnes_list)
    new_population_set = mate_population(progenitor_list)
    
    mutated_pop = mutate_population(new_population_set)
end = timer()
print("Time: {}".format(end - start))

0 5498096.71621259 5609130.468561614 15/12/22 14:57
100 5487468.388502843 5639961.009440341 15/12/22 15:00
200 5473944.364723589 5618479.043202379 15/12/22 15:03
300 5501527.230701186 5628927.1683054725 15/12/22 15:06
400 5397015.294461456 5621456.899105823 15/12/22 15:09
Time: 901.8330256


In [20]:
best_solution[1]

5345588.80420102